In [1]:
import numpy as np
import numpy.linalg as npl
import numpy as np
from scipy.sparse import diags

In [2]:
def modNewtonBcktrck(x0, f, gradf, hessf, alpha0, kmax, tolgrad, c1, rho, btmax):
    xk = x0.copy()
    k = 0
    
    # Store the sequence of iterates
    xseq = [xk.copy()]
    grad_xk = gradf(xk)
    gradfk_norm = npl.norm(grad_xk)
    n = len(x0) # Dimensione del problema
    while k < kmax and gradfk_norm > tolgrad:
        H = hessf(xk)
        # --- INIZIO MODIFICA PER NEWTON MODIFICATO ---
        # Cerchiamo di rendere H definita positiva aggiungendo tau * I
        tau = 0
        beta = 1e-3  # Valore iniziale piccolo se tau deve essere aumentato
        while True:
            # Creiamo la matrice modificata: B = H + tau * I
            B = H + tau * np.eye(n)
            #IMPORTANTE contare le volte che viene modificata la matrice
            try:
                # Proviamo la fattorizzazione di Cholesky.
                # Se funziona, B è definita positiva.
                # Se lancia LinAlgError, B non è definita positiva.
                np.linalg.cholesky(B)
                
                # Se siamo qui, la matrice è OK. Risolviamo il sistema.
                # B * pk = -grad
                pk = -np.linalg.solve(B, grad_xk)
                break # Usciamo dal loop di correzione tau
                
            except np.linalg.LinAlgError:
                # La matrice non è definita positiva. Aumentiamo tau.
                if tau == 0:
                    tau = beta
                else:
                    tau = 2 * tau # Aumentiamo rapidamente tau
        # --- FINE MODIFICA ---

        # Backtracking Line Search
        alpha_k = getAlpha(f, gradf, xk, pk, alpha0, rho, c1, btmax)
        
        # Aggiornamento xk
        xk = xk + alpha_k * pk
        
        # Calcolo gradienti per la prossima iterazione
        grad_xk = gradf(xk)
        gradfk_norm = npl.norm(grad_xk)
        
        k += 1
        xseq.append(xk.copy())
        
    xseq = np.column_stack(xseq)
    return xk, f(xk), gradfk_norm, k, xseq

def getAlpha(f, grad_f, x, p, alpha0=1.0, rho=0.5, c=1e-4, btmax=20):
    k = 0
    satisfyed = False
    fx = f(x)
    grad_fx = grad_f(x)
    
    # Pre-calcoliamo il termine lineare per efficienza
    grad_dot_p = np.dot(grad_fx, p)
    
    while k < btmax and not satisfyed:
        alpha = alpha0 * (rho ** k)
        # Condizione di Armijo
        if f(x + alpha * p) <= fx + c * alpha * grad_dot_p:
            satisfyed = True
            print(k)
        else:
            k += 1
    
    return alpha if satisfyed else alpha0 * (rho ** btmax)

In [3]:
def broyden_tridiagonal(x):
    """
    Calcola la funzione obiettivo tridiagonale di Broyden.
    
    F(x) = 0.5 * sum(f_k(x)^2)
    
    Dove:
    f_k(x) = (3 - 2*x_k)*x_k - x_{k-1} - 2*x_{k+1} + 1
    con x_0 = x_{n+1} = 0
    """
    x = np.asarray(x)
    
    # Array con zero all'inizio e alla fine
    # x_pad diventerà: [0, x_1, x_2, ..., x_n, 0], 
    # serve per poi fare i calcoli con vettori piuttosto che cilco
    x_pad = np.pad(x, (1, 1), mode='constant', constant_values=0)
    
    # Definiamo i vettori spostati per il calcolo vettorizzato
    x_k = x_pad[1:-1]       # x_k corrisponde agli elementi centrali (l'input originale)
    x_prev = x_pad[0:-2]    # x_{k-1} corrisponde agli elementi spostati a sinistra
    x_next = x_pad[2:]      # x_{k+1} corrisponde agli elementi spostati a destra
    
    # in questo modo:
    # x_prev [0, 1, 2, 3]
    # x_k    [1, 2, 3, 4]
    # x_next [2, 3, 4, 0]
    
    
    # Per non fare cicli calcolo del vettore f_k per tutti i k da 1 a n simultaneamente
    f_vec = (3 - 2 * x_k) * x_k - x_prev - 2 * x_next + 1
    
    # Calcolo finale di F(x)
    F_x = 0.5 * np.sum(f_vec**2)
    
    return F_x

In [ ]:
import numpy as np
from scipy.sparse import diags

def hessian_tridiagonal_sparse(func, x, h=1e-5):
    n = len(x)
    # Pre-allochiamo i 3 array (Main, Upper, Lower)
    main_diag = np.zeros(n)
    upper_diag = np.zeros(n-1) # Lunghezza n-1
    lower_diag = np.zeros(n-1) # Lunghezza n-1
    f_x = func(x)
    # 1. Calcolo DIAGONALE (Main)
    for i in range(n):
        orig_i = x[i]
        x[i] = orig_i + h; fp = func(x)
        x[i] = orig_i - h; fm = func(x)
        # Formula calcolo diagonale hessiana
        main_diag[i] = (fp - 2*f_x + fm) / (h**2)
        x[i] = orig_i
    # 2. Calcolo SOPRA-DIAGONALE (Upper) e SOTTO (Lower)
    # Calcoliamo solo Upper e copiamo in Lower per simmetria
    for i in range(n - 1):
        j = i + 1
        orig_i = x[i]
        orig_j = x[j]
        # Formula mista a 4 punti
        x[i]=orig_i+h; x[j]=orig_j+h; f_pp=func(x)
        x[i]=orig_i+h; x[j]=orig_j-h; f_pm=func(x)
        x[i]=orig_i-h; x[j]=orig_j+h; f_mp=func(x)
        x[i]=orig_i-h; x[j]=orig_j-h; f_mm=func(x)
        val = (f_pp - f_pm - f_mp + f_mm) / (4 * h**2)
        upper_diag[i] = val 
        lower_diag[i] = val # Simmetria
        x[i] = orig_i; x[j] = orig_j
    # 3. Creazione della Matrice Sparsa (Formato DIA)
    # offsets: 0 è la diagonale, +1 quella sopra, -1 quella sotto
    # Nota: upper_diag e lower_diag devono essere "allineati" per la funzione diags,
    # ma spesso è più facile passarli come lista di array corretti.
    
    # Scipy vuole che gli array abbiano la stessa lunghezza della dimensione massima
    # o gestisce automaticamente se passi array più corti con l'offset corretto.
    
    return diags([lower_diag, main_diag, upper_diag], [-1, 0, 1], shape=(n, n))


x0 = -1 * np.ones(10) # Dimensione 5
print(x0, broyden_tridiagonal(x0))
H_sparse = hessian_tridiagonal_sparse(broyden_tridiagonal, x0)

print("Matrice Sparsa (rappresentazione):")
print(H_sparse)

print("\nMatrice Completa (solo per visualizzare):")
print(H_sparse.toarray())

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.] 10.5
Matrice Sparsa (rappresentazione):
<DIAgonal sparse matrix of dtype 'float64'
	with 28 stored elements (3 diagonals) and shape (10, 10)>
  Coords	Values
  (1, 0)	-21.000001737547787
  (2, 1)	-21.000001737547787
  (3, 2)	-21.000006178439886
  (4, 3)	-21.000006178439886
  (5, 4)	-21.000006178439886
  (6, 5)	-21.000006178439886
  (7, 6)	-21.000006178439886
  (8, 7)	-21.000006178439886
  (9, 8)	-21.000001737547787
  (0, 0)	58.00000479894151
  (1, 1)	57.99998703537312
  (2, 2)	58.00000479894151
  (3, 3)	58.00000479894151
  (4, 4)	58.00000479894151
  (5, 5)	58.00000479894151
  (6, 6)	58.00000479894151
  (7, 7)	57.99998703537312
  (8, 8)	57.99996927180472
  (9, 9)	65.0000231416925
  (0, 1)	-21.000001737547787
  (1, 2)	-21.000001737547787
  (2, 3)	-21.000006178439886
  (3, 4)	-21.000006178439886
  (4, 5)	-21.000006178439886
  (5, 6)	-21.000006178439886
  (6, 7)	-21.000006178439886
  (7, 8)	-21.000006178439886
  (8, 9)	-21.000001737547787

Matrice 

In [5]:
# --- ESEMPIO DI UTILIZZO ---
n = 10
x0 = -1 * np.ones(n)
risultato = broyden_tridiagonal(x0)

print(f"Dimensione n: {n}")
print(f"Punto iniziale x: {x0}")
print(f"Risultato F(x): {risultato}")

Dimensione n: 10
Punto iniziale x: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Risultato F(x): 10.5


In [7]:
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# 1. DEFINIZIONE DEL PROBLEMA (Funzione, Gradiente, Hessiana)
# ---------------------------------------------------------

def get_f_vec(x):
    """Calcola il vettore dei residui f_k(x)"""
    x = np.asarray(x)
    # Padding: x_0=0, x_{n+1}=0
    x_pad = np.pad(x, (1, 1), mode='constant', constant_values=0)
    x_k = x_pad[1:-1]
    x_prev = x_pad[0:-2]
    x_next = x_pad[2:]
    # f_k = (3 - 2x_k)x_k - x_{k-1} - 2x_{k+1} + 1
    return (3 - 2 * x_k) * x_k - x_prev - 2 * x_next + 1

def get_jacobian(x):
    """Calcola la matrice Jacobiana J di f(x). J è tridiagonale."""
    n = len(x)
    J = np.zeros((n, n))
    
    # Diagonale principale: derivata parziale di f_k rispetto a x_k
    # d(f_k)/d(x_k) = 3 - 4*x_k
    diag_val = 3 - 4 * x
    np.fill_diagonal(J, diag_val)
    
    # Diagonale inferiore: derivata parziale di f_k rispetto a x_{k-1}
    # d(f_k)/d(x_{k-1}) = -1
    idx_lower = np.arange(1, n)
    J[idx_lower, idx_lower - 1] = -1
    
    # Diagonale superiore: derivata parziale di f_k rispetto a x_{k+1}
    # d(f_k)/d(x_{k+1}) = -2
    idx_upper = np.arange(0, n - 1)
    J[idx_upper, idx_upper + 1] = -2
    
    return J

def broyden_func(x):
    """Funzione Obiettivo Scalare F(x)"""
    f_vec = get_f_vec(x)
    return 0.5 * np.sum(f_vec**2)

def broyden_grad(x):
    """Gradiente di F(x) = J^T * f"""
    f_vec = get_f_vec(x)
    J = get_jacobian(x)
    return J.T @ f_vec

def broyden_hess(x):
    """
    Hessiana di F(x).
    H = J^T * J + sum(f_k * Hessian(f_k))
    
    Nota: Hessian(f_k) è molto sparsa. La derivata seconda di f_k 
    rispetto a x_k è -4. Le miste sono 0.
    Quindi il secondo termine è una matrice diagonale.
    """
    f_vec = get_f_vec(x)
    J = get_jacobian(x)
    
    # Termine Gauss-Newton
    H_gn = J.T @ J
    
    # Termine del secondo ordine (S = sum f_k * nabla^2 f_k)
    # nabla^2 f_k ha solo l'elemento (k,k) = -4 diverso da zero
    S = np.diag(-4 * f_vec)
    
    return H_gn + S

# ---------------------------------------------------------
# 3. ESECUZIONE DEL TEST
# ---------------------------------------------------------

# Parametri del problema
n = 10000  # Dimensione
x0 = -1 * np.ones(n) # Punto iniziale standard (-1, -1, ...)

# Esecuzione
xk_final, f_final, grad_norm, iterations, xseq = modNewtonBcktrck(
    x0, 
    broyden_func, 
    broyden_grad, 
    broyden_hess,
    alpha0=1.0, 
    kmax=100, 
    tolgrad=1e-6, 
    c1=1e-4, 
    rho=0.5, 
    btmax=50
)

print(f"\nRisultato Finale:")
print(f"Iterazioni totali: {iterations}")
print(f"Punto di minimo trovato (primi 5 elementi): {xk_final[:5]}...")
print(f"Valore funzione al minimo: {f_final}")
print(f"Norma gradiente finale: {grad_norm}")

0
0
0
0
0

Risultato Finale:
Iterazioni totali: 5
Punto di minimo trovato (primi 5 elementi): [-0.57076119 -0.68191013 -0.70248602 -0.70626058 -0.70695185]...
Valore funzione al minimo: 1.5481925847692407e-17
Norma gradiente finale: 1.7909357354915685e-08


In [ ]:
# ---------------------------------------------------------
# 4. GRAFICO CONVERGENZA
# ---------------------------------------------------------
history_f = [broyden_func(xseq[:, i]) for i in range(xseq.shape[1])]
history_g = [npl.norm(broyden_grad(xseq[:, i])) for i in range(xseq.shape[1])]

fig, ax1 = plt.subplots(figsize=(10, 5))

color = 'tab:blue'
ax1.set_xlabel('Iterazioni')
ax1.set_ylabel('Valore Funzione F(x)', color=color)
ax1.plot(history_f, color=color, marker='o', label='F(x)')
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_yscale('log')

ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Norma Gradiente ||∇F||', color=color) 
ax2.plot(history_g, color=color, linestyle='--', marker='x', label='||Grad||')
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yscale('log')

plt.title('Convergenza Metodo Newton Modificato (Broyden Tridiagonal)')
fig.tight_layout()
plt.show()